# Advent of Code Puzzles
[Advent of Code 2019](https://adventofcode.com/2019) | [reddit/adventofcode](https://www.reddit.com/r/adventofcode/)

In [3]:
from pathlib import Path
import pandas as pd
import numpy as np

## [Day 03](https://adventofcode.com/2021/day/03): Crossed Wires 
### Part I
- **Unknown**: The [Manhattan distance](https://en.wikipedia.org/wiki/Taxicab_geometry) from the central port to the closest intersection.
- **Data**: Two wire's paths
- **Condition**:
    - A wire crossing with itself doesn't count.
- **Plan**:
    - Parse paths
    - Learn to analyze and represent paths



In [16]:
def get_xy(move):
    """Transform move to axis + directed lenght"""
    what = move[0]
    how = int(move[1:])
    if what in ['R', 'L']:
        ax = 'x'
        di = how if what == 'R' else -how
    elif what in ['U','D']:
        ax = 'y'
        di = how if what == 'U' else -how
    else:
        ax, di = "NA" , 0
        print(f"get_xy() warning: Unknown move {move}")

    return (ax, di)

for move in ['R11', 'U12', 'L20', 'D2', 'X12']:
    print(get_xy(move))

('x', 11)
('y', 12)
('x', -20)
('y', -2)
get_xy() warning: Unknown move X12
('NA', 0)


In [17]:
def parse(file):
    """Parse input to two list with an ax and a direction"""
    txt = Path(file).read_text().strip()
    wires = [line.split(',') for line in txt.split('\n')]
    for wire in wires:
        for m in range(len(wire)):
            wire[m] = get_xy(wire[m])
    return wires
wires1 = parse('example1.txt')
print(wires1)
wiresx = parse('input.txt')
print(len(wiresx[0]), len(wiresx[1]))

[[('x', 8), ('y', 5), ('x', -5), ('y', -3)], [('y', 7), ('x', 6), ('y', -4), ('x', -4)]]
301 301


The `parse()` function returns two lists of movements. I need to clarify the terms and concepts.

This representation can be friendly for determining the shape of matrices and initializing two grids of wires.

After adding two grids it will be possible to find the Manhattan distances from the central port to all intersections and return the minimum distance.


### Part II
- **Unknown**: 
- **Data**: 
- **Condition**:
- **Plan**:
